In [1]:
import pandas as pd
import numpy as np
import argparse
import time
from copy import deepcopy
import os
import warnings

from sklearn.metrics import classification_report, f1_score, accuracy_score,precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import WeightedRandomSampler
import math
from tqdm import tqdm
warnings.filterwarnings('ignore')


def makemap(df0, r, e=1e-6):
    df = np.array(deepcopy(df0))
    n, m = df.shape
    f = int(m / r)
    df = np.hstack([df[:, j * f:(j + 1) * f] /
                    df[:, j * f:(j + 1) * f].sum(1).reshape(-1, 1) for j in range(r)])
    f_mean = df.mean(1)
    data = np.zeros((n, 1, m, m), dtype=np.float)
    for x in range(n):
        for i in range(m):
            for j in range(m):
                if i < j:
                    data[x, 0, i, j] = df[x, i] - df[x, j]
                if i == j:
                    data[x, 0, i, j] = df[x, i] - f_mean[x]
                if i > j:
                    data[x, 0, i, j] = np.tanh(np.log2((df[x, i] + e) / (df[x, j] + e)))
    return data.round(3)


class channel_attention(nn.Module):
    def __init__(self, channel, ratio=2):
        super(channel_attention, self).__init__()
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // ratio, False),
            nn.ReLU(),
            nn.Linear(channel // ratio, channel, False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, h, w = x.size()
        max_pool_out = self.max_pool(x).view([b, c])
        avg_pool_out = self.avg_pool(x).view([b, c])

        max_fc_out = self.fc(max_pool_out)
        avg_fc_out = self.fc(avg_pool_out)

        out = max_fc_out + avg_fc_out
        out = self.sigmoid(out).view([b, c, 1, 1])

        return out * x


class spacial_attention(nn.Module):
    def __init__(self, kernel_size=3):
        super(spacial_attention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, 1, padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_pool_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_pool_out = torch.mean(x, dim=1, keepdim=True)
        pool_out = torch.cat([max_pool_out, avg_pool_out], dim=1)

        out = self.conv(pool_out)
        out = self.sigmoid(out)

        # print(out)

        return out * x


class DeepSP_nocnn(nn.Module):
    def __init__(self, fraction, out_channels, channel=32, ratio=2, kernel_size=3):
        super(DeepSP_nocnn, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3, 1, 1),
                                   nn.BatchNorm2d(32), nn.ReLU(),
               #                   nn.MaxPool2d(kernel_size=2)
                                  )

        self.channel_attention = channel_attention(channel, ratio)
        self.spacial_attention = spacial_attention(kernel_size)
        self.fc1 = nn.Sequential(
            nn.Linear(fraction ** 2, 512),
            nn.ReLU(),
            nn.Dropout(p=0.3))

#         self.fc1 = nn.Sequential(
#             nn.Linear(32 * (fraction // 2) ** 2, 512),
#             nn.ReLU(),
#             nn.Dropout(p=0.3))

        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.3))

        self.fc3 = nn.Linear(256, out_channels)

    def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
 #       x = self.conv3(x)
#         x = self.channel_attention(x)
#         x = self.spacial_attention(x)
        x = self.fc1(x.view(x.size(0), -1))
        x = self.fc2(x)
        x = self.fc3(x)
        return x



class MultiFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=0):
        super(MultiFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, outputs, targets):
        if self.alpha is None and self.gamma == 0:
            focal_loss = F.cross_entropy(outputs, targets)

        elif self.alpha is not None and self.gamma == 0:
            self.alpha = torch.tensor(self.alpha) / torch.tensor(self.alpha).sum()
            self.alpha = self.alpha.to(outputs)
            focal_loss = F.cross_entropy(outputs, targets, weight=self.alpha)

        elif self.alpha is None and self.gamma != 0:
            ce_loss = F.cross_entropy(outputs, targets, reduction='none')
            p_t = torch.exp(-ce_loss)
            focal_loss = ((1 - p_t) ** self.gamma * ce_loss).mean()

        else:
            self.alpha = torch.tensor(self.alpha) / torch.tensor(self.alpha).sum()
            self.alpha = self.alpha.to(outputs)
            ce_loss = F.cross_entropy(outputs, targets, reduction='none')
            p_t = torch.exp(-ce_loss)
            ce_loss = F.cross_entropy(outputs, targets, weight=self.alpha, reduction='none')
            focal_loss = ((1 - p_t) ** self.gamma * ce_loss).mean()

        return focal_loss


def train(model, train_loader, val_db, index,  weight):
    x_val, y_val = val_db
    sm = nn.Softmax(dim=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2)
    loss_func = MultiFocalLoss(alpha=weight, gamma=2)
    val_f1score_best = 0.0
    for epoch in range(1, epochs + 1):
        model.train()
        for i, (x_batch, y_batch) in enumerate(train_loader):
            out = model(x_batch)
            loss = loss_func(out, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_out = model(x_val)
            loss = loss_func(val_out, y_val)
            val_loss = loss.item()
            y_valprob = sm(val_out)
            y_valscore, y_valpred = torch.max(y_valprob, 1)
            val_f1score = f1_score(y_val, y_valpred, average='macro')
            val_acc = accuracy_score(y_val, y_valpred)

#             if epoch % nepoch == 0:
#                 print('[{}/{}] val_loss: {} | f1score: {} | accuracy: {}'
#                       .format(epoch, epochs, val_loss, val_f1score, val_acc))

            if val_f1score_best < val_f1score:
                bestepoch = epoch
                val_f1score_best = val_f1score
                val_acc_best = val_acc
                val_loss_best = val_loss
                y_valprob_best = y_valprob
                torch.save(model, 'model{}.pkl'.format(index + 1))

#     print('bech epoch [{}/{}] | val_loss: {} | f1score: {} | accuracy: {}'
#           .format(bestepoch, epochs, val_loss_best, val_f1score_best, val_acc_best))

    return y_valprob_best.numpy()


def cross_validation(x, y):
    y_trainprob = np.zeros((y.shape[0], len(set(target))))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cv_index = np.zeros(y.shape[0])
    for index, (train_index, val_index) in enumerate(skf.split(x, y)):
#         print('Cross validation: ', index + 1)
        x_train, x_val = x[train_index], x[val_index]
        y_train, y_val = y[train_index], y[val_index]
        train_db = TensorDataset(x_train, y_train)
        weight = 1 / np.bincount(y_train)
        train_loader = DataLoader(train_db,
                                  batch_size=batch_size,
                                  num_workers=0,
                                  shuffle=True)
        val_db = (x_val, y_val)
        torch.manual_seed(seed)
        model = DeepSP_nocnn(fraction, len(set(target)))
        y_trainprob[val_index] = train(model, train_loader, val_db, index, weight)
        cv_index[val_index] = index + 1

    return y_trainprob, cv_index


def predict(x_test):
    sm = nn.Softmax(dim=1)
    y_testprob_result = np.zeros((x_test.shape[0], len(set(target))))
    for index in range(n_splits):
        model = torch.load('model{}.pkl'.format(index + 1))
        model.eval()
        with torch.no_grad():
            y_testprob_result += sm(model(x_test)).numpy() / n_splits
    return y_testprob_result


In [6]:
datafile = 'orre2019.csv'
rep = 1

n_splits = 5
batch_size = 64
epochs = 100
lr = 1e-3
l2 = 0.0
data = pd.read_csv(datafile, index_col=0)



test_results = []
test_f1scores = []
test_accs = []
test_precisions = []
test_recalls = []
test_size = 0.2

data = data[data['markers'] != 'unknown']
target = data.markers
class_mapping = {label: idx for idx, label in enumerate(sorted(set(target)))}
inv_class_mapping = {v: k for k, v in class_mapping.items()}

train_results = []
train_f1scores = []
train_accs = []
train_precisions = []
train_recalls = []
train_quadlosses = []


test_results = []
test_f1scores = []
test_accs = []
test_precisions = []
test_recalls = []
test_quadlosses = []
for seed in tqdm(range(100)):
    test_set = pd.concat([data[data.markers == i].sample(math.ceil(data[data.markers == i].shape[0] * test_size))
        for i in pd.unique(data.markers)],axis=0)
    train_set = data[~data.index.isin(test_set.index)]
    train_data = train_set.drop('markers', axis=1)
    train_y = train_set.markers

    test_data = test_set.drop('markers', axis=1)
    test_y = test_set.markers
  
    x_train = torch.tensor(makemap(train_data.values, rep), dtype=torch.float)
    y_train = torch.tensor(train_y.map(class_mapping).values)
    fraction = x_train.shape[2]
    y_trainprob, cv_index = cross_validation(x_train, y_train)
    train_result = pd.DataFrame(data=y_trainprob, index=train_set.index, columns=list(class_mapping.keys()))
    train_pred_array = train_result.values
    
    train_result['true_label'] = train_y
    train_result['pred_label'] = y_trainprob.argmax(1)
    train_result['pred_label'] = train_result['pred_label'].map(inv_class_mapping)
    train_result['pred_prob'] = y_trainprob.max(1)
    train_result['cv_index'] = cv_index
    train_result['repeat'] = seed + 1
    train_result = train_result.reset_index()
    train_results.append(train_result)
    train_f1scores.append(f1_score(train_result.true_label,
                                  train_result.pred_label,average='macro'))
    train_precisions.append(precision_score(train_result.true_label,
                                           train_result.pred_label,average='macro'))
    train_recalls.append(recall_score(train_result.true_label,
                                     train_result.pred_label,average='macro'))
    train_accs.append(accuracy_score(train_result.true_label, train_result.pred_label))
    train_true_array = pd.get_dummies(train_result.true_label)[class_mapping.keys()].values
    train_quadlosses.append(sum(sum(np.dot((train_true_array - train_pred_array),
                                           (train_true_array - train_pred_array).T))) / train_result.shape[0])
    
    
    x_test = torch.tensor(makemap(test_data.values, rep), dtype=torch.float)
    y_testprob = predict(x_test)
    
    test_result = pd.DataFrame(data=y_testprob, index=test_set.index, columns=list(class_mapping.keys()))
    test_pred_array = test_result.values
    test_result['true_label'] = test_y
    test_result['pred_label'] = y_testprob.argmax(1)
    test_result['pred_label'] = test_result['pred_label'].map(inv_class_mapping)
    test_result['pred_prob'] = y_testprob.max(1)
    test_result['repeat'] = seed + 1
    test_result = test_result.reset_index()
    test_results.append(test_result)
    
    test_f1scores.append(f1_score(test_result.true_label,
                                  test_result.pred_label,average='macro'))
    test_precisions.append(precision_score(test_result.true_label,
                                           test_result.pred_label,average='macro'))
    test_recalls.append(recall_score(test_result.true_label,
                                     test_result.pred_label,average='macro'))
    test_accs.append(accuracy_score(test_result.true_label, test_result.pred_label))
    test_true_array = pd.get_dummies(test_result.true_label)[class_mapping.keys()].values
    test_quadlosses.append(sum(sum(np.dot((test_true_array - test_pred_array),
                                           (test_true_array - test_pred_array).T))) / test_result.shape[0])

train_results = pd.concat(train_results)
test_results = pd.concat(test_results)

100%|██████████| 100/100 [1:48:35<00:00, 65.15s/it]


In [7]:
name = datafile.split('.')[0]
train_results.to_csv(f'{name}_DeepSP_nocnn_train_results.csv')
test_results.to_csv(f'{name}_DeepSP_nocnn_test_results.csv')

In [8]:
trainresult = pd.DataFrame({
    'data': datafile.split('.')[0],
    'method': 'WithoutCNN',
    'f1score': train_f1scores,
    'precision': train_precisions,
    'recall': train_recalls,
    'accuracy': train_accs,
    'quadloss': train_quadlosses
})
trainresult.to_csv(name+'DeepSP_nocnn_train.csv', index=None)
trainresult.mean()

f1score      0.759583
precision    0.757470
recall       0.766493
accuracy     0.765910
quadloss     4.308817
dtype: float64

In [9]:
testresult = pd.DataFrame({
    'data': datafile.split('.')[0],
    'method': 'WithoutCNN',
    'f1score': test_f1scores,
    'precision': test_precisions,
    'recall': test_recalls,
    'accuracy': test_accs,
    'quadloss': test_quadlosses
})
testresult.to_csv(name+'DeepSP_nocnn_test.csv', index=None)
testresult.mean()

f1score      0.751655
precision    0.754129
recall       0.757444
accuracy     0.759720
quadloss     1.216974
dtype: float64